# Sandbox

For playing around and testing. The contents aren't meant to makes sense afterwards or even run in order.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
batch = [
    ([1, 2 , 3], (3, 3, "A")),
    ([3, 3 , 4], (3, 3, "b")),
    ([2, 2 , 5], (3, 3, "c")),
]